In [11]:
# import modules
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, LeakyReLU, Input, BatchNormalization, Softmax, Reshape, Conv2DTranspose, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import RandomNormal
import matplotlib.pyplot as plt

In [12]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [13]:
num_train_real_data_items = train_images.shape[0]
num_test_real_data_items = test_images.shape[0]

In [14]:
# activation_funcion = Activation("tanh")
# activation_funcion = Activation("sigmoid")
activation_funcion = LeakyReLU(alpha=0.2)
use_normalisation = True
use_dropout = True
dropout = 0.2

In [15]:
encoder_kernal_nums = [20,]
encoder_kernal_sizes = [3]
assert len(encoder_kernal_sizes) == len(encoder_kernal_nums)

encoder_dense_layer_neurons = [100, 100]
latent_space_size = 100
assert latent_space_size == encoder_dense_layer_neurons[-1]

In [16]:
model_encoder = Sequential()

model_encoder.add(Input((28, 28, 1)))

for kernal_size, kernal_num in zip(encoder_kernal_sizes, encoder_kernal_nums):
    model_encoder.add(
        Conv2D(
            kernal_num,
            kernel_size=(kernal_size, kernal_size), 
            kernel_initializer=RandomNormal(stddev=0.02)
        )
    )

    if use_dropout:
        model_encoder.add(Dropout(dropout))

    if use_normalisation:
        model_encoder.add(BatchNormalization())

    model_encoder.add(activation_funcion)

model_encoder.add(
    Flatten()
)

for num_neurons in encoder_dense_layer_neurons:
    model_encoder.add(
        Dense(num_neurons)
    )

    model_encoder.add(activation_funcion)

In [17]:
model_encoder.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 20)        200       
                                                                 
 dropout_1 (Dropout)         (None, 26, 26, 20)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 26, 26, 20)        80        
 chNormalization)                                                
                                                                 
 leaky_re_lu_1 (LeakyReLU)   multiple                  0         
                                                                 
 flatten_1 (Flatten)         (None, 13520)             0         
                                                                 
 dense_6 (Dense)             (None, 100)               1352100   
                                                      

In [18]:
# decoder model structure based on emulating encoder structure in reverse

dencoder_dense_layer_neurons = [10_000, 135_200]
dencoder_kernal_nums = [20,]
dencoder_kernal_sizes = [3]
convolutional_layer_shape = (26, 26, 4, 1)

In [19]:


model_encoder = Sequential()


model_encoder.add(Input(latent_space_size))

for num_neurons in reversed(dencoder_dense_layer_neurons):
    model_encoder.add(
        Dense(num_neurons)
    )
    model_encoder.add(activation_funcion)

model_encoder.add(
    Reshape(convolutional_layer_shape)
)

for kernal_size, kernal_num in zip(reversed(dencoder_kernal_sizes), reversed(dencoder_kernal_nums)):
    model_encoder.add(
        Conv2DTranspose(
            kernal_num,
            kernel_size=(kernal_size, kernal_size), 
            kernel_initializer=RandomNormal(stddev=0.02)
        )
    )

    if use_dropout:
        model_encoder.add(Dropout(dropout))

    if use_normalisation:
        model_encoder.add(BatchNormalization())

    model_encoder.add(activation_funcion)


model_encoder.add(Reshape((28, 28, 1)))


ValueError: Exception encountered when calling layer "reshape" (type Reshape).

total size of new array must be unchanged, input_shape = [10000], output_shape = [26, 26, 4, 1]

Call arguments received by layer "reshape" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 10000), dtype=float32)